In [ ]:
import os

ENDWITHS = 'Manga109'

NOTEBOOK_DIR = os.getcwd()
if not NOTEBOOK_DIR.endswith(ENDWITHS):
    raise ValueError(f"Not in correct dir, expect end with {ENDWITHS}, but got {NOTEBOOK_DIR} instead")

BASE_DIR = os.path.join(NOTEBOOK_DIR,'..', '..','..','..','..')
import sys
import os

# Add the code directory to path
sys.path.insert(0, os.path.join(BASE_DIR, 'code'))
from pipeline.SegmentationModels.YoloSeg import YoloSeg


In [ ]:
from SegDataset import MangaBubbleDataset, gather_json
from SegDataloader import manga_collate_fn, MangaBubbleDataLoader
from torchvision import transforms
import json
import pandas as pd
import torch 

YOLO_MODEL_PATH = os.path.join(BASE_DIR, "models","bubble-detection","YOLO_Ultralytics","super_yolo.pt")
BATCH_SIZE= 8
IMAGE_DIR= os.path.join(BASE_DIR, "data","Manga109_released_2023_12_07","images")
MASK_DIR= os.path.join(BASE_DIR, "data","MangaSegmentation","jsons_processed")
IMG_SIZE= (1024, 1024)
TEST_JSON_FILE= os.path.join(NOTEBOOK_DIR, "test.json")
SPLIT_SUMMARY_CSV= os.path.join(BASE_DIR,"code","utils","train_test_split_manga109.csv")
device = "cuda:1" if torch.cuda.is_available() else "cpu"
model= YoloSeg(YOLO_MODEL_PATH, device= device, verbose= False)
model.load_model()

In [ ]:
split_sum= pd.read_csv(SPLIT_SUMMARY_CSV)
series_list_df= split_sum[split_sum['train_test_split']=='test']
series_list= series_list_df['volumes'].reset_index(drop=True)

In [ ]:
test_json = gather_json(series_list, MASK_DIR)

with open(TEST_JSON_FILE, "w+") as f:
    json.dump(test_json, f, indent=2)

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()])

train_loader = MangaBubbleDataLoader(TEST_JSON_FILE, IMAGE_DIR, IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False,num_workers=0, transform= transform)

In [ ]:

from ClassEvaluator import ClassEvaluator

e= ClassEvaluator(train_loader, model)
res= e.evaluate()

In [ ]:
e.print_results()